In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import numpy as np
import plotly.graph_objects as go
from datetime import timedelta
import requests
import seaborn as sns
import warnings

warnings.filterwarnings("ignore")

# Задание №1

In [2]:
from scipy.stats import norm

# Среднее время доставки
mean_delivery_time = 45

# Стандартное отклонение
std_dev = 5

# Время ожидания, для которого мы хотим рассчитать вероятность
waiting_time = 55

# Рассчитываем Z-оценку (стандартизированное значение)
z_score = (waiting_time - mean_delivery_time) / std_dev

# Рассчитываем вероятность с использованием функции распределения
probability = 1 - norm.cdf(z_score)

print(f"Вероятность ждать более {waiting_time} минут: {probability:.4f}")

Вероятность ждать более 55 минут: 0.0228


# Задание № 2

In [4]:
import statsmodels.api as sm

successes_group1 = 0.61 * 3500  # количество успехов в первой группе
nobs_group1 = 3500      # общее количество наблюдений в первой группе

successes_group2 = 0.63 * 3500  # количество успехов во второй группе
nobs_group2 = 3500      # общее количество наблюдений во второй группе

# проведение теста
stat, p_value = sm.stats.proportions_ztest([successes_group1, successes_group2], [nobs_group1, nobs_group2])

# Вывод результатов
print(f"Z-статистика: {stat}")
print(f"P-значение: {p_value}")

# Проверка статистической значимости при уровне значимости 0.05
if p_value < 0.05:
    print("Различие статистически значимо.")
else:
    print("Различие не статистически значимо.")

Z-статистика: -1.7236987907469261
P-значение: 0.08476223344508192
Различие не статистически значимо.


Таким образом, при тестировании долей в двух выборках мы получаем, что различие статистически не значимо на уровне доверия 95%. Следовательно, при выбранном уровне значимости 0.05 мы можем сказать, что нет достаточных доказательств, чтобы утверждать, что наличие онбординга приводит к статистически значимому улучшению retention day 1. Значит, отчёту доверять нельзя.

В чём может быть причина? Что можно предпринять?

1. Возможно, стоит продлить эксперимент: нельзя заглядывать в эксперимент и делать финальные выводы до полного его окончания;

2. Запустить эксперимент с большей выборкой: если размер выборки недостаточен, тест может не быть достаточно чувствительным;

3. Более детально проудмать имплементируемые изменения, собрать обратную связь от пользователей.


# Задание № 3

In [5]:
# Читаем данные
users = pd.read_csv('https://gitlab.com/hse_mar/mar211f/raw/main/data/hw_2/users.csv')
auth = pd.read_csv('https://gitlab.com/hse_mar/mar211f/raw/main/data/hw_2/auth.csv')

In [6]:
users.head()

,user_id,install_date,onboarding,start_screen
0,96298fda,2019-10-22,test,control
1,a1ebd508,2019-10-14,control,control
2,d454a789,2019-10-21,test,control
3,97843c4d,2019-10-12,test,control
4,4ca19c7c,2019-10-15,test,test


In [7]:
auth.head()

,user_id,login_date
0,0002c7b0,2019-10-23
1,0002db79,2019-10-10
2,0002db79,2019-10-11
3,0002db79,2019-10-11
4,000739f8,2019-10-12


In [8]:
# мёрджим
data = pd.merge(left=users, right=auth, how='left', on='user_id')

In [9]:
data['install_date'] = pd.to_datetime(data['install_date'])
data['login_date'] = pd.to_datetime(data['login_date'])

In [10]:
data.head()

,user_id,install_date,onboarding,start_screen,login_date
0,96298fda,2019-10-22,test,control,2019-10-22
1,96298fda,2019-10-22,test,control,2019-10-23
2,a1ebd508,2019-10-14,control,control,2019-10-14
3,a1ebd508,2019-10-14,control,control,2019-10-15
4,d454a789,2019-10-21,test,control,2019-10-21


In [19]:
# протестируем результаты онбоардинга
onboarding = data.groupby(['user_id', 'onboarding']).agg({'install_date': 'min', 'login_date': 'max'})
onboarding['days'] = (onboarding['login_date'] -  onboarding['install_date']).dt.days
onboard = onboarding.reset_index().groupby('onboarding').agg({'days': 'mean', 'user_id': 'count'}).rename(columns={'user_id': 'users_count', 'days': 'ret_share'})

In [20]:
group_1 = onboard.iloc[0]
group_2 = onboard.iloc[1]

In [ ]:
successes_group1 = group_1['ret_share'] * group_1['users_count'] # количество успехов в первой группе
nobs_group1 = group_1['users_count'] # общее количество наблюдений в первой группе

successes_group2 = group_2['ret_share'] * group_2['users_count']  # количество успехов во второй группе
nobs_group2 = group_2['users_count'] # общее количество наблюдений во второй группе

# проведение теста (не забываем выставить односторонюю альтернативу в соответствии с заданием)
stat, p_value = sm.stats.proportions_ztest([successes_group1, successes_group2], [nobs_group1, nobs_group2], alternative='smaller')

# вывод результатов
print(f"Z-статистика: {stat}")
print(f"P-значение: {p_value}")

# проверка статистической значимости при уровне значимости 0.05
if p_value < 0.05:
    print("Различие статистически значимо.")
else:
    print("Различие не статистически значимо.")

Z-статистика: -1.1029980828566583
P-значение: 0.135013998852676
Различие не статистически значимо.


Таким образом, можем заключить, что ни на уровне доверия, равным 0.95, ни на 0.9 полученные различия не являются значимыми. Следовательно, имеющиеся данные свидетельствуют об отсутствии различий между выборками.

In [ ]:
# проинтерпретируем результаты изменения стартового экрана
screen = data.groupby(['user_id', 'start_screen']).agg({'install_date': 'min', 'login_date': 'max'})
screen['days'] = (screen['login_date'] -  screen['install_date']).dt.days
screen_ = screen.reset_index().groupby('start_screen').agg({'days': 'mean', 'user_id': 'count'}).rename(columns={'user_id': 'users_count', 'days': 'ret_share'})

In [ ]:
screen_

,ret_share,users_count
start_screen,,
control,0.31085,10645
test,0.32591,10552


In [ ]:
group_1 = screen_.iloc[0]
group_2 = screen_.iloc[1]

In [ ]:
successes_group1 = group_1['ret_share'] * group_1['users_count'] # количество успехов в первой группе
nobs_group1 = group_1['users_count'] # общее количество наблюдений в первой группе

successes_group2 = group_2['ret_share'] * group_2['users_count']  # количество успехов во второй группе
nobs_group2 = group_2['users_count'] # общее количество наблюдений во второй группе

# проведение теста (не забываем выставить односторонюю альтернативу в соответствии с заданием)
stat, p_value = sm.stats.proportions_ztest([successes_group1, successes_group2], [nobs_group1, nobs_group2], alternative='smaller')

# вывод результатов
print(f"Z-статистика: {stat}")
print(f"P-значение: {p_value}")

# проверка статистической значимости при уровне значимости 0.05
if p_value < 0.05:
    print("Различие статистически значимо.")
else:
    print("Различие не статистически значимо.")

Z-статистика: -2.353341113692693
P-значение: 0.00930277866341907
Различие статистически значимо.


В случае с тестированием нового дизайна стартового окна приложения получили, что результаты статистически значимы на **любом разумном уровне значимости**(с) П.К. Катышев:)

-> изменение помогло в более длительном удержании пользователей.

In [21]:
# проведём тестирование для двух факторов 

from statsmodels.formula.api import ols

onboarding_start = data.groupby(['user_id', 'onboarding', 'start_screen']).agg({'install_date': 'min', 'login_date': 'max'})

onboarding_start['days'] = (onboarding_start['login_date'] -  onboarding_start['install_date']).dt.days

onboarding_start = onboarding_start.reset_index()


model = ols('days ~ C(onboarding) + C(start_screen) + C(onboarding):C(start_screen)', data=onboarding_start).fit()

anova_table = sm.stats.anova_lm(model, typ=2)

print(anova_table)

                                    sum_sq       df         F    PR(>F)
C(onboarding)                     0.266022      1.0  1.226295  0.268141
C(start_screen)                   1.203820      1.0  5.549319  0.018497
C(onboarding):C(start_screen)     0.904390      1.0  4.169019  0.041182
Residual                       4597.422660  21193.0       NaN       NaN


Выводы:

-p-value для C(onboarding) равно 0.268, что больше 0.05. Это означает, что нет значимой разницы в среднем значении зависимой переменной между уровнями фактора onboarding, независимо от фактора start_screen.

-p-value для C(start_screen) равно 0.018, что меньше 0.05. Это означает, что есть значимая разница в среднем значении зависимой переменной между уровнями фактора start_screen, независимо от фактора onboarding.

-p-value для C(onboarding):C(start_screen) равно 0.041, что меньше 0.05. Это означает, что есть значимый взаимодействующий эффект между факторами onboarding и start_screen, то есть влияние одного фактора зависит от уровня другого фактора.

С учётом полученных результатов предложим следующие рекомендации:

1. Улучшение онбоардинга:

- Провести детальный анализ данных и обратной связи пользователей, чтобы выявить конкретные слабые места в существующем онбординге;

- Основываясь на анализе, внести изменения в онбординг с целью повышения его эффективности;

- Внимательно следить за метриками удержания после каждого обновления и итеративно вносить коррективы.

2. Сегментация юзеров

- Разделить пользователей на сегменты и проанализировать, какие группы пользователей наиболее подвержены улучшениям в онбординге;

- Проанализировать целесообразность настройки онбординга для разных категорий пользователей.

# Задание № 4

План A/B-теста для внедрения темной темы в MS Office:

1. Возможные гипотезы:
   - Гипотеза 1: Пользователи будут привлечены темной темой, так как она создает более комфортное визуальное восприятие и снижает нагрузку на глаза.
   - Гипотеза 2: Пользователи могут быть неуверенными в использовании темной темы, опасаясь возможных проблем с читаемостью или непривычности интерфейса.

2. Возможные метрики:
   - Метрика 1: Количество пользователей, активирующих темную тему.
   - Метрика 2: Удовлетворенность пользователями, измеряемая опросами или обратной связью.

3. Определить размер выборки:
   - Размер выборки будет зависеть от ожидаемого эффекта и уровня статистической значимости. Необходимо провести предварительный расчет, чтобы определить минимальный размер выборки для достижения достаточной мощности теста.

4. Разбить на группы:
   - Разделить пользователей на две группы: контрольную и экспериментальную.
   - Контрольная группа будет использовать стандартную светлую тему MS Office.
   - Экспериментальная группа будет использовать новую темную тему.

5. Показать пользователям:
   - Внедрить темную тему в MS Office только для пользователей в экспериментальной группе.
   - Обеспечить возможность пользователям переключаться между светлой и темной темами.

6. Проанализировать:
   - Собрать данные о количестве активаций темной темы и обратной связи от пользователей.
   - Сравнить результаты между контрольной и экспериментальной группами.
   - Использовать статистические методы для оценки статистической значимости различий между группами.

7. Обнародовать результаты:
   - Представить результаты A/B-теста, включая статистическую значимость и практическую значимость.
   - Проанализировать обратную связь пользователей и учесть ее при принятии решения о внедрении темной темы для всех пользователей MS Office.

Риски:
- Риск 1: Негативная реакция пользователей на новую тему, что может привести к снижению удовлетворенности и использованию MS Office.
- Риск 2: Технические проблемы или несовместимость новой темы с определенными версиями MS Office или операционными системами.

Критерии оценивания:
- Критерий 1: Увеличение количества активаций темной темы в экспериментальной группе по сравнению с контрольной группой.
- Критерий 2: Положительная обратная связь от пользователей, выражающая удовлетворенность и комфорт при использовании темной темы.

Дополнительные шаги, которые можно выполнить до этапа анализа данных:
- Провести исследование и анализ запросов пользователей в коммьюнити, чтобы лучше понять их потребности и предпочтения относительно темной темы.
- Провести прототипирование и тестирование темной темы с небольшой группой пользователей, чтобы собрать предварительную обратную связь и внести необходимые улучшения перед проведением A/B-теста.